In [26]:
%matplotlib inline

import re
import pandas as pd
import os


In [27]:
df = pd.read_csv('200_500_str.csv')

In [28]:
df.drop(df[pd.isnull(df['sentences_200_str'])].index, inplace=True)

In [29]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np

In [30]:
MAX_SEQUENCE_LENGTH = 200
EMBEDDING_DIM = 200


tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['sentences_200_str'])
sequences = tokenizer.texts_to_sequences(df['sentences_200_str'])


In [31]:
word_index = tokenizer.word_index

In [32]:
all_data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
labels = to_categorical(np.asarray(df['label']))
print('Shape of data tensor:', all_data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (73065, 200)
Shape of label tensor: (73065, 14)


In [33]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val = train_test_split(all_data,labels,test_size=0.3,stratify=labels)

In [34]:
x_train.shape,y_train.shape

((51145, 200), (51145, 14))

In [35]:
from keras.layers import Dense, Input, Flatten, Dropout
from keras.layers import LSTM, Embedding
from keras.models import Sequential

model = Sequential()
model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, 
          input_length=MAX_SEQUENCE_LENGTH))
model.add(LSTM(200, dropout=0.2, recurrent_dropout=0.2))
model.add(Dropout(0.2))
model.add(Dense(labels.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])
print(model.metrics_names)
model.summary()

['loss', 'acc']
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 200, 200)          82824200  
_________________________________________________________________
lstm_3 (LSTM)                (None, 200)               320800    
_________________________________________________________________
dropout_3 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 14)                2814      
Total params: 83,147,814
Trainable params: 83,147,814
Non-trainable params: 0
_________________________________________________________________


In [36]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, History 
model_path ='/home/jovyan/jt071-group23/jt071080/rnn_TFIDF_200.h5'
#if not os.path.exists(model_path):
    #os.makedirs(model_path)

checkpoint = ModelCheckpoint(model_path, monitor='val_acc', save_best_only=True, verbose=1)
earlystop = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
history = History()


model.fit(x_train, y_train, 
          validation_data=(x_val, y_val), 
          epochs=50, 
          batch_size=128,
          callbacks = [earlystop, checkpoint, history]
         )

Train on 51145 samples, validate on 21920 samples
Epoch 1/50
51072/51145 [============================>.] - ETA: 0s - loss: 1.4022 - acc: 0.5513
Epoch 00001: val_acc improved from -inf to 0.76031, saving model to /home/jovyan/jt071-group23/jt071080/rnn_TFIDF_200.h5
51145/51145 [==============================] - 155s 3ms/step - loss: 1.4017 - acc: 0.5515 - val_loss: 0.7771 - val_acc: 0.7603
Epoch 2/50
51072/51145 [============================>.] - ETA: 0s - loss: 0.7074 - acc: 0.7908
Epoch 00002: val_acc improved from 0.76031 to 0.80985, saving model to /home/jovyan/jt071-group23/jt071080/rnn_TFIDF_200.h5
51145/51145 [==============================] - 155s 3ms/step - loss: 0.7073 - acc: 0.7908 - val_loss: 0.6935 - val_acc: 0.8099
Epoch 3/50
51072/51145 [============================>.] - ETA: 0s - loss: 0.5314 - acc: 0.8481
Epoch 00003: val_acc improved from 0.80985 to 0.82130, saving model to /home/jovyan/jt071-group23/jt071080/rnn_TFIDF_200.h5
51145/51145 [=============================

KeyboardInterrupt: 

In [37]:
print(history.history)

{'val_loss': [0.77709022699481378, 0.69347922714957355, 0.62464546147924271, 0.59039410504981549, 0.59463659237771138, 0.62880898687961329], 'val_acc': [0.76031021897810214, 0.80985401459854012, 0.82130474452554747, 0.8402372262773723, 0.84219890510948903, 0.83713503649635035], 'loss': [1.4016888055252252, 0.70728701048203146, 0.53141781120009413, 0.41795214921081619, 0.33761442046336082, 0.27002788842604275], 'acc': [0.55152996386329334, 0.790790888673225, 0.84813764783361578, 0.88241274809135772, 0.90585589992509397, 0.92499755599046807]}


In [38]:
import json

with open('first200_bs128.txt', 'a') as file:
     file.write(json.dumps(history.history))